In [1]:
import requests
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean

In [2]:
# Definir las URL
url_base = 'https://raw.githubusercontent.com/IvanLetteri/MTA-KDD-19/master/'
url_mta = url_base + 'datasetLegitimate33featues.csv'
url_leg = url_base + 'datasetMalware33featues.csv'

# Rutas relativas para guardar los archivos CSV
data_dir = './data'
os.makedirs(data_dir, exist_ok=True)
mta_file_path = os.path.join(data_dir, 'datasetLegitimate33featues.csv')
leg_file_path = os.path.join(data_dir, 'datasetMalware33featues.csv')

In [3]:
# Función para descargar archivos si no existen
def download_if_not_exists(url, file_path):
    if not os.path.exists(file_path):
        print(f"Descargando {file_path}...")
        response = requests.get(url)
        with open(file_path, 'w') as f:
            f.write(response.text)
        print(f"{file_path} guardado exitosamente.")
    else:
        print(f"{file_path} ya existe.")

# Descargar los archivos sólo si no existen
download_if_not_exists(url_mta, mta_file_path)
download_if_not_exists(url_leg, leg_file_path)

./data/datasetLegitimate33featues.csv ya existe.
./data/datasetMalware33featues.csv ya existe.


In [11]:
# Iniciar una sesión de Spark
spark = SparkSession.builder \
    .appName("MalwareAnalysis") \
    .config("spark.eventLog.gcMetrics.youngGenerationGarbageCollectors", "G1") \
    .config("spark.eventLog.gcMetrics.oldGenerationGarbageCollectors", "G1") \
    .getOrCreate()

In [12]:
# Cargar en PySpark
dfMTA = spark.read.csv(mta_file_path, header=True, inferSchema=True)
dfLEG = spark.read.csv(leg_file_path, header=True, inferSchema=True)

In [13]:
# Unir ambos DataFrames y mostrar estadísticas descriptivas
df = dfMTA.union(dfLEG)
df.show(5)

+-------------------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+--------------------+------------------+------------------+-------------------+-----+
|        FinFlagDist|        SynFlagDist|        RstFlagDist|       PshFlagDist|       AckFlagDist|          DNSoverIP|          TCPoverIP|           UDPoverIP|              MaxLen|             MinLen|          StdDevLen|             AvgLen|             MaxIAT|              MinIAT|              AvgIAT|         AvgWinFlow|        

In [14]:
# Observaciones donde el tráfico es anómalo
anomalous_data = df.filter(col('label') == 1)
anomalous_data.show(5)

+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+--------------------+------------------+--------------------+------------------+-----+
|        FinFlagDist|         SynFlagDist|        RstFlagDist|         PshFlagDist|        AckFlagDist|          DNSoverIP|          TCPoverIP|           UDPoverIP|              MaxLen|             MinLen|           StdDevLen|             AvgLen|              MaxIAT|              MinIAT|             AvgIAT|         AvgWi

In [16]:
df.describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|summary|         FinFlagDist|         SynFlagDist|         RstFlagDist|         PshFlagDist|         AckFlagDist|           DNSoverIP|           TCPoverIP|           UDPoverIP|              MaxLen|              MinLen|           StdDevLen|              AvgLen|              Max

In [17]:
# Agrupar por etiqueta y calcular estadísticas para cada grupo
df.groupBy("label").agg(
    mean("MaxLen").alias("Promedio MaxLen"),
    mean("AvgLen").alias("Promedio AvgLen"),
    mean("MinLen").alias("Promedio MinLen"),
    mean("MaxIAT").alias("Promedio MaxIAT"),
    mean("AvgIAT").alias("Promedio AvgIAT"),
    mean("MinIAT").alias("Promedio MinIAT")
).show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|label|     Promedio MaxLen|     Promedio AvgLen|     Promedio MinLen|     Promedio MaxIAT|     Promedio AvgIAT|     Promedio MinIAT|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0.0|  0.3736062775836766| 0.28185258137141234|-0.25698957470952066| 0.36456352917253804|  0.4200333907253716|-0.04183748829954543|
|  1.0|-0.32853293005371176|-0.24784876474784265| 0.22598538782865102|-0.32058113478285777|-0.36935889157937973| 0.03679004728195187|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [18]:
# Diferencia entre las columnas 'MaxLen' y 'MinLen'
df = (df
    # Diferencia entre la longitud máxima y mínima de paquetes
    .withColumn('Len_Diff', col('MaxLen') - col('MinLen'))
    # Razón entre el tiempo promedio de llegada entre paquetes y el mínimo
    .withColumn('IAT_Ratio', col('AvgIAT') / col('MinIAT'))
    # Diferencia entre el flujo total y el recibido
    .withColumn('Flow_Diff', col('FlowLEN') - col('FlowLENrx'))
    .select('MaxLen', 'MinLen', 'Len_Diff', 'AvgIAT', 'MinIAT', 'IAT_Ratio',
    'FlowLEN', 'FlowLENrx', 'Flow_Diff'))

# Mostrar las nuevas columnas
df.show(5)

+--------------------+-------------------+-------------------+--------------------+--------------------+------------------+------------------+-------------------+------------------+
|              MaxLen|             MinLen|           Len_Diff|              AvgIAT|              MinIAT|         IAT_Ratio|           FlowLEN|          FlowLENrx|         Flow_Diff|
+--------------------+-------------------+-------------------+--------------------+--------------------+------------------+------------------+-------------------+------------------+
|  0.7174699003847623|   1.57360970710837|-0.8561398067236076| -1.0938387855451712|-0.07973699430297695|13.718083997358967| 2.861360204033125| 0.2575641129201597| 2.603796091112965|
|  0.7174699003847623|0.17977198110019849| 0.5376979192845639| -1.1060634026076386|-0.07973789568827833|13.871238926740736| 2.329183130318988| 21.255366145115577|-18.92618301479659|
|  0.7174699003847623|   1.57360970710837|-0.8561398067236076|  -1.079806855989545|-0.0797

In [19]:
# Mostrar sólo registros con una diferencia de longitudes mayor a 1
df.filter(col('Len_Diff') > 1).show(5)

+--------------------+-------------------+------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------------------+
|              MaxLen|             MinLen|          Len_Diff|             AvgIAT|              MinIAT|          IAT_Ratio|            FlowLEN|           FlowLENrx|           Flow_Diff|
+--------------------+-------------------+------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------------------+
|  0.6911578779951258|-1.0286089015799738|1.7197667795750995| 1.0749857465616293| -0.0797243749997373|-13.483777659783115|  0.722175262392494|-0.03253740001234...|  0.7547126624048419|
|-0.02323296055329...|-1.0286089015799738|1.0053759410266765| 1.5721787676424483|-0.06288118370762845| -25.00237233052785|-0.2886650141157917|-0.04825636245126483|-0.24040865166452688|
|  0.7174699003847623|-1.0286089015799738|1.7460788019647362|-1.10589422337

In [20]:
# Filtrar los flujos donde la razón entre llegadas de paquetes es mayor a un
# umbral
df.filter(col("IAT_Ratio") > 10).show(5)

+------------------+-------------------+-------------------+-------------------+--------------------+------------------+------------------+-------------------+------------------+
|            MaxLen|             MinLen|           Len_Diff|             AvgIAT|              MinIAT|         IAT_Ratio|           FlowLEN|          FlowLENrx|         Flow_Diff|
+------------------+-------------------+-------------------+-------------------+--------------------+------------------+------------------+-------------------+------------------+
|0.7174699003847623|   1.57360970710837|-0.8561398067236076|-1.0938387855451712|-0.07973699430297695|13.718083997358967| 2.861360204033125| 0.2575641129201597| 2.603796091112965|
|0.7174699003847623|0.17977198110019849| 0.5376979192845639|-1.1060634026076386|-0.07973789568827833|13.871238926740736| 2.329183130318988| 21.255366145115577|-18.92618301479659|
|0.7174699003847623|   1.57360970710837|-0.8561398067236076| -1.079806855989545|-0.07973789568827833|13.5

In [21]:
# Filtrar los flujos donde la razón entre llegas máxima es mayor a un umbral
df.filter(col("MaxIAT") > 1).show(5)

+-------------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+
|             MaxLen|             MinLen|            Len_Diff|             AvgIAT|              MinIAT|          IAT_Ratio|            FlowLEN|           FlowLENrx|          Flow_Diff|
+-------------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+
| 0.7174699003847623|   1.57360970710837| -0.8561398067236076| 0.9058979525996136|-0.07972978325175212|-11.362102286659683|0.46052779887410206|-0.01576129746599...|0.47628909634009636|
|-1.2628791423639254|-1.0286089015799738|-0.23427024078395164| 1.8382377367086349|  0.7488439081889976|  2.454767564517183|-0.9538844632105857|-0.05008838306041833|-0.9037960801501674|
|-1.2628791423639254|-1.0286089015799738|-0.23427024078395164| 1.8159714757